<a href="https://colab.research.google.com/github/anmutechsupport/AD-biomarker-optimization/blob/main/BOED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install adopy

In [ ]:
import numpy as np
from tqdm import tqdm 
import scipy
from scipy.stats import truncnorm

In [ ]:
from adopy import Task

task = Task(name='My New Experiment',  # Name of the task (optional)
            designs = ['x1', 'x2'],    # Labels of design variables
            responses = ['choice'])    # Labels of response variables

In [ ]:
import numpy as np
from scipy.stats import bernoulli

def calculate_loglik(x1, x2, b0, b1, b2, choice):
    """A function to compute the probability of a positive response."""
    logit = b0 + x1 * b1 + x2 * b2
    p_obs = 1. / (1 + np.exp(-logit))

    return bernoulli.logpmf(choice, p_obs)

In [ ]:
def get_simulated_response(model, design, choice):
  # Probability to choose the LL option
  logmf = model.compute(design['x1'], design['x2'], -3, 2, 3, choice)
  # Compute a random binary choice
  return logmf

In [ ]:
def get_truncated_normal(mean=0.5, sd=0.5, low=0, upp=1):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [ ]:
from adopy import Model

model = Model(name='My Logistic Model',   # Name of the model (optional)
              params=['b0', 'b1', 'b2'],  # Labels of model parameters
              func=calculate_loglik,
              task=task)      # A log likelihood function

In [ ]:
import numpy as np

grid_design = {
    'x1': np.linspace(0, 50, 20),    # 100 grid points within [0, 50]
    'x2': np.linspace(-20, 30, 20),  # 100 grid points within [-20, 30]
}

grid_param = {
    'b0': np.linspace(-5, 5, 20),  # 100 grid points within [-5, 5]
    'b1': np.linspace(-5, 5, 20),  # 100 grid points within [-5, 5]
    'b2': np.linspace(-5, 5, 20),  # 100 grid points within [-5, 5]
}

grid_response = {
    'choice': [0, 1]  # Binary choice
}

In [ ]:
from adopy import Engine

engine = Engine(model=model,                  # Model object
                task=task,                    # Task object
                grid_design=grid_design,      # grid for design variables
                grid_param=grid_param,        # grid for model parameters
                grid_response=grid_response)  # grid for response variables

In [ ]:
# Compute an optimal design based on the ADO
design = engine.get_design()
design = engine.get_design('optimal')

# Compute a randomly chosen design, as is typically done in non-ADO experiments
rand_design = engine.get_design('random')

In [ ]:
# Get a response from a participant using your own code
response = 1

In [ ]:
# Update the engine with the design and the corresponding response
engine.update(design, response)
print(engine.get_design('optimal'))

{'x1': 0.0, 'x2': 30.0}


In [ ]:
engine.reset()
NUM_TRIAL = 4 # number of trials
patients = [{'biomarkers': {'x1': np.random.uniform(0, 50), 'x2': np.random.uniform(-20, 30)}} for x in range(10)]
interim = 16

print(len(patients))

for analysis in tqdm(range(NUM_TRIAL*interim)):

    opt_design = engine.get_design()
    print(opt_design)

    for patient in patients:
      # Compute an optimal design for the current trial

      # Get a response using the optimal design
      # patient['response'] = get_simulated_response(model, opt_design, engine.grid_response['choice'])  # Using users' own codes
      patient['response'] = get_truncated_normal().rvs()  # Using users' own codes
      # print(patient['response'])

    bio_pos = []
    bio_neg = []
    for patient in patients:
      if patient["biomarkers"]['x1']+patient["biomarkers"]['x2']>opt_design['x1']+opt_design['x2']:
        bio_pos.append(patient['response'])
      else:
        bio_neg.append(patient['response'])

    statistic, p_value = scipy.stats.ttest_ind(bio_pos, bio_neg)
    print(bio_neg)
    print(bio_pos)
    print(p_value)

    if p_value<0.05:
      break
    
    engine.update(opt_design, p_value) #change these to have design variables and p-value response
  



  0%|          | 0/64 [00:00<?, ?it/s]

10
{'x1': 50.0, 'x2': 1.0526316165924072}
[0.48770884725302693, 0.22402725390551997, 0.047400163156691655, 0.521670176772097, 0.752699822941564, 0.005197402138639229, 0.6671034713586377, 0.7726145114135028, 0.2988054149480933]
[0.9999185423744235]
nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3622: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  2%|▏         | 1/64 [00:00<00:33,  1.89it/s]

{'x1': 0.0, 'x2': 30.0}
[0.030936466504779603, 0.870070966346757, 0.7346562198986626, 0.2673104992420747, 0.17619528565390874, 0.538751618087588]
[0.806180327202136, 0.414973049776653, 0.13363607054602272, 0.17767745942295238]
0.8047966723609279


  3%|▎         | 2/64 [00:01<00:32,  1.90it/s]

...

 97%|█████████▋| 62/64 [00:34<00:01,  1.92it/s]

{'x1': 2.6315789222717285, 'x2': 24.736841201782227}
[0.6782991577353263, 0.9598703058350577, 0.11831718003017094, 0.848285187402199]
[0.421594402188874, 0.9178761816245524, 0.18178204183283003, 0.8079472764168922, 0.28488742973839654, 0.39203576435670506]
0.49657277405325384


 98%|█████████▊| 63/64 [00:34<00:00,  1.90it/s]

{'x1': 2.6315789222717285, 'x2': 24.736841201782227}
[0.6479739842024256, 0.08457894951353473, 0.9023210412994753, 0.7684618031460885]
[0.4792345529449748, 0.5156126599201232, 0.6224013299110454, 0.26401360201311364, 0.8340218388128592, 0.6827014744626805]
0.8472866582594852


100%|██████████| 64/64 [00:35<00:00,  1.81it/s]


In [ ]:
optimal_biomarkers = engine.get_design()
 
print(optimal_biomarkers)
print(len(patients))
print(engine.post_mean)
i = 0 
for patient in patients:
  comb = patient['biomarkers']['x1'] + patient['biomarkers']['x2']
  if comb > optimal_biomarkers['x2'] + optimal_biomarkers['x1']:
    i += 1 
    print(i)

{'x1': 2.6315789222717285, 'x2': 24.736841201782227}
10
b0   -4.433045
b1   -0.789251
b2    0.263158
Name: Posterior mean, dtype: float32
1
2
3
4
5
6
